In [1]:
import json

import numpy as np
import pandas as pd

In [2]:
def gen_html(contribution_dict, orcid, name):
    html = """
    <!doctype html>
    <title>{orcid}</title>
    <h1>{name} ({orcid})</h1>
    <br/>
    """.format(orcid=orcid, name=name)

    for section in contribution_dict.keys():
        html += '<br/><h2>{0}</h2>'.format(section.upper())
        for app in contribution_dict[section].keys():
            metrics = list(contribution_dict[section][app].keys())
            if len(metrics) == 1:
                val = contribution_dict[section][app][metrics[0]]
                if not np.isnan(float(val)) and float(val) != 0:
                    html += '<br/>{0}: {1}'.format(app, val)
            else:                
                html += '<br/><h3>{0}</h3>'.format(app)
                html += '<br/><ul>'
                for metric in metrics:
                    val = contribution_dict[section][app][metric]
                    if not np.isnan(float(val)) and float(val) != 0:
                        html += '<br/><li>{0}: {1}</li>'.format(metric, val)
                html += '<br/></ul>'
    return html

In [3]:
df = pd.read_csv('all_contributions.csv', index_col='orcid')

with open('database/ids.json', 'r') as fo:
    ids_dict = json.load(fo)

# Make dictionary with contribution info, indexed by ORCID

In [4]:
contrib = {}
for col in df.columns:
    values = df[col].values
    values = values[~np.isnan(values)]
    if np.array_equal(values, values.astype(int)):
        df[col] = df[col].fillna(-1)
        df[col] = df[col].astype(int)
        df[col] = df[col].astype(str)
        df[col] = df[col].replace('-1', np.nan)
    
    subsections = col.split('_____')
    section = subsections[0]
    subsubsections = subsections[1].split('____')
    app, metric = subsubsections
    app = app.replace('___', '/').replace('__', '/')
    
    for orcid in ids_dict.keys():
        if orcid not in contrib.keys():
            contrib[orcid] = {}
        
        if section not in contrib[orcid].keys():
            contrib[orcid][section] = {}

        if app not in contrib[orcid][section].keys():
            contrib[orcid][section][app] = {}

        contrib[orcid][section][app][metric] = df.loc[orcid, col]


# Make html file for each ORCID

In [6]:
for id_ in contrib.keys():
    temp = contrib[id_]
    name = ids_dict[id_]['name']
    html = gen_html(temp, id_, name)
    
    with open('profiles/{0}.html'.format(id_), 'w') as fo:
        fo.write(html)